In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
!export HF_HOME=/mnt/storage/tmp
!echo $HF_HOME

/mnt/storage/tmp


In [3]:
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from accelerate import Accelerator
from huggingface_hub import notebook_login

In [4]:
notebook_login()

In [7]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-13b-chat-hf"

# The instruction dataset to use
dataset_name = "Andrei481/hakurei-cleaned-ro"

# Fine-tuned model name
new_model = "Llama-2-13b-chat-hakurei-ro-v0.3"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./llama-2-13b-chat-hakurei-ro-v0.3"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 8 # CHANGE THIS TO 8 ON VM

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8 # CHANGE THIS TO 8 ON VM

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
# device_map = {"": Accelerator().process_index}

device_map = 'auto'
cache_directory = "/mnt/storage/tmp"

In [8]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    cache_dir=cache_directory
)
print(model)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=cache_directory)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 18610
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize the accelerator to handle multi-GPU training
accelerator = Accelerator()

# Adjust batch size for multi-GPU training
training_arguments.per_device_train_batch_size = per_device_train_batch_size

# Distribute the model and data across GPUs
#model = accelerator.prepare(model)
#dataset = accelerator.prepare(dataset)



/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [9]:
  EOS_TOKEN = tokenizer.eos_token
  BOS_TOKEN = tokenizer.bos_token
  PAD_TOKEN = tokenizer.pad_token
  print(EOS_TOKEN)
  print(BOS_TOKEN)
  print(PAD_TOKEN)


</s>
<s>
▁***


In [10]:
prompt = """Mai jos este prezentată o instrucțiune care descrie o sarcină, însoțită de o intrare care oferă un context suplimentar. Scrieți un răspuns care să completeze în mod corespunzător cererea. Trebuie să răspundeți numai în limba română.

### Instrucțiune:
{}

### Intrare:
{}

### Răspuns:
{}"""

chat_prompt = """<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Dacă o întrebare nu are niciun sens sau nu este coerentă din punct de vedere factual, explicați de ce în loc să răspundeți la ceva incorect. Dacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie sa răspundeți doar în limba română.
<</SYS>>

{} [/INST]
{}"""

def format_prompt(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN # IMPORTANT - WITHOUT THIS, GENERATION WILL GO ON FOREVER
        texts.append(text)
    return { "text" : texts }

def format_prompt_chat(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is not None:
            text = chat_prompt.format(instruction + " " + input, output) + EOS_TOKEN # IMPORTANT - WITHOUT THIS, GENERATION WILL GO ON FOREVER
        else:
            text = chat_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }
    

In [11]:
dataset = dataset.map(format_prompt_chat, batched=True)
dataset[20]

Map:   0%|          | 0/69385 [00:00<?, ? examples/s]

{'instruction': 'Transcrieți înregistrarea în text.',
 'input': 'Înregistrare despre istoria Waterloo',
 'output': 'Bătălia de la Waterloo, desfășurată la 18 iunie 1815, a fost ultima bătălie majoră a războaielor napoleoniene. \nEa a marcat sfârșitul domniei lui Napoleon Bonaparte. \nBătălia a avut loc între forțele franceze ale lui Napoleon Bonaparte și o forță aliată formată din trupe britanice, olandeze, belgiene și prusace, condusă de Ducele de Wellington. \nS-a desfășurat în apropierea orașului Waterloo, în fosta Olanda austriacă, în prezent Belgia de astăzi. \nBătălia a fost o înfrângere pentru forțele lui Napoleon și a pus capăt dominației lui Napoleon asupra Franței. \nNapoleon însuși a fost exilat la Sfânta Elena, unde avea să moară în 1821. \nBătălia a marcat sfârșitul dominației franceze în Europa și începutul unei perioade de pace în regiune. \nA fost, de asemenea, ultima bătălie majoră a Războaielor napoleoniene și este amintită ca fiind una dintre cele mai importante bătă

In [12]:
# Initialize the trainer with the updated training arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
torch.cuda.empty_cache()

/mnt/storage/.conda/envs/finetune_ro/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/69385 [00:00<?, ? examples/s]

In [13]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.242000
50,2.650600
75,2.004600
100,1.706800
125,1.128300
150,0.721200
175,0.871700
200,0.561000
225,0.777000
250,0.522700


TrainOutput(global_step=26022, training_loss=0.44825264784716395, metrics={'train_runtime': 303287.8504, 'train_samples_per_second': 0.686, 'train_steps_per_second': 0.086, 'total_flos': 2.3721239492909875e+18, 'train_loss': 0.44825264784716395, 'epoch': 3.0})

In [14]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [18]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(chat_prompt.format(
        "Care dintre urmatoarele este capitala Romaniei? Budapesta, Viena, Stuttgart, Berlin, Londra", # instruction + input
        "", # output - leave this blank for generation!
    ))
# result = pipe("<s>[INST] Care este capitala Romaniei? [/INST]")
print(result[0]['generated_text'])

<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Dacă o întrebare nu are niciun sens sau nu este coerentă din punct de vedere factual, explicați de ce în loc să răspundeți la ceva incorect. Dacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie sa răspundeți doar în limba română.
<</SYS>>

Care dintre urmatoarele este capitala Romaniei? Budapesta, Viena, Stuttgart, Berlin, Londra [/INST]
București


In [17]:
inputs = tokenizer(
[
    prompt.format(
        "Care este capitala Romaniei?", # instruction
        "", # input
        "" + EOS_TOKEN, # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Mai jos este prezentată o instrucțiune care descrie o sarcină, însoțită de o intrare care oferă un context suplimentar. Scrieți un răspuns care să completeze în mod corespunzător cererea. Trebuie să răspundeți numai în limba română.\n\n### Instrucțiune:\nCare este capitala Romaniei?\n\n### Intrare:\n\n\n### Răspuns:\n</s><s> nobody\n\n#include "nobody.h"\n\n#include <stdio.h>\n#include <stdlib.h>\n#include <string.h>\n#include <unistd.h>\n#include <sys/types.h>\n#include <sys']

In [19]:
# Empty VRAM
del model
del pipe
del trainer

gc.collect()
gc.collect()

0

In [ ]:
# Saving the model to Huggingface

In [20]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token_id = 18610
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
model.push_to_hub("Andrei481/Llama-2-13b-chat-hakurei-ro-v0.3", check_pr=True, private=False)

tokenizer.push_to_hub("Andrei481/Llama-2-13b-chat-hakurei-ro-v0.3",check_pr=True, private=False)


pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Andrei481/Llama-2-13b-chat-hakurei-ro-v0.3/commit/846baaa52637fbdf742f9b3b333ae3887b430beb', commit_message='Upload tokenizer', commit_description='', oid='846baaa52637fbdf742f9b3b333ae3887b430beb', pr_url=None, pr_revision=None, pr_num=None)